In [6]:
!pip install pygame


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 5.3 MB/s  0:00:02a 0:00:010:00:0102


In [4]:
import sys
print(sys.version)

3.11.13 (main, Jun  5 2025, 13:12:00) [GCC 11.2.0]


In [ ]:
import cv2
import time
import os
import numpy as np
import pygame


try:
    import mediapipe as mp
except Exception as e:
    raise ImportError("Could not import mediapipe. Install with 'pip install mediapipe'.\n" + str(e))

pygame.mixer.init()
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils


CHORD_KEYS = {
    ord('1'): 'A', ord('2'): 'B', ord('3'): 'C', ord('4'): 'D',
    ord('5'): 'E', ord('6'): 'F', ord('7'): 'G',
    ord('a'): 'A', ord('b'): 'B', ord('c'): 'C', ord('d'): 'D',
    ord('e'): 'E', ord('f'): 'F', ord('g'): 'G',
}
chords = ['A','B','C','D','E','F','G']
selected_chord = 'A'

CHORD_IMG_DIR = 'chords'
SOUNDS_DIR = 'sounds'


STRUM_TOP_RATIO = 0.35      
STRUM_BOTTOM_RATIO = 0.75   
NUM_STRINGS = 6


import cv2
import time
import os
import numpy as np
import pygame


try:
    import mediapipe as mp
except Exception as e:
    raise ImportError("Could not import mediapipe. Install with 'pip install mediapipe'.\n" + str(e))

pygame.mixer.init()
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils


CHORD_KEYS = {
    ord('1'): 'A', ord('2'): 'B', ord('3'): 'C', ord('4'): 'D',
    ord('5'): 'E', ord('6'): 'F', ord('7'): 'G',
    ord('a'): 'A', ord('b'): 'B', ord('c'): 'C', ord('d'): 'D',
    ord('e'): 'E', ord('f'): 'F', ord('g'): 'G',
}
chords = ['A','B','C','D','E','F','G']
selected_chord = 'E'

CHORD_IMG_DIR = 'chords'
SOUNDS_DIR = 'sounds'

STRUM_TOP_RATIO = 0.35
STRUM_BOTTOM_RATIO = 0.75
NUM_STRINGS = 6


STRING_HIT_THRESHOLD_RATIO = 0.6


STRING_COOLDOWN = 0.08


STRING_WAV_MAPPING = {
    0: 'E3.wav',   
    1: 'A3.wav',
    2: 'D4.wav',
    3: 'G4.wav',
    4: 'B4.wav',
    5: 'E5.wav',   
}


STRING_COLORS = [
    (200, 80, 20),
    (0, 200, 0),
    (20, 20, 200),
    (200, 200, 0),
    (200, 0, 200),
    (0, 200, 200),
]


FINGERTIP_LANDMARKS = [4, 8, 12, 16, 20]


preloaded_sounds = {}
for idx, fname in STRING_WAV_MAPPING.items():
    path = os.path.join(SOUNDS_DIR, fname)
    if os.path.exists(path):
        try:
            preloaded_sounds[idx] = pygame.mixer.Sound(path)
        except Exception as e:
            print(f"[WARN] Could not load sound {path}: {e}")
    else:
        print(f"[WARN] Missing sound file: {path}")


if len(preloaded_sounds) < NUM_STRINGS:
    print("[WARN] Some string sounds are missing. Make sure sounds/ contains E3.wav, A3.wav, D4.wav, G4.wav, B4.wav, E5.wav")


def load_chord_image(ch):
    path = os.path.join(CHORD_IMG_DIR, f"{ch}.png")
    if os.path.exists(path):
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        if img is None:
            return None
        if img.shape[2] == 4:
            alpha = img[:, :, 3] / 255.0
            img_rgb = img[:, :, :3].astype(np.float32)
            for c in range(3):
                img[:, :, c] = (alpha * img_rgb[:, :, c] + (1 - alpha) * 255).astype(np.uint8)
            img = img[:, :, :3]
        return img
    return None

chord_images = {c: load_chord_image(c) for c in chords}


CHORD_POSITIONS = {
    "A": [None, 0, 2, 2, 2, 0],
    "C": [None, 3, 2, 0, 1, 0],
    "D": [None, None, 0, 2, 3, 2],
    "E": [0, 2, 2, 1, 0, 0],
    "G": [3, 2, 0, 0, 0, 3],
    "Em": [0, 2, 2, 0, 0, 0],
    "Am": [None, 0, 2, 2, 1, 0],
    
}


SONGS = {
    "Simple": [("C", 2.0), ("G", 2.0), ("Am", 2.0), ("F", 2.0)],
    "Twinkle": [("C", 1.5), ("C", 1.5), ("G", 1.5), ("G", 1.5),
                ("A", 1.5), ("A", 1.5), ("G", 3.0)],
}
current_song = None
song_name = None
song_start_time = None
current_chord_index = 0
song_mode = False


last_play_times = [0.0] * NUM_STRINGS

def play_string(string_index):
    now = time.time()
    if now - last_play_times[string_index] < STRING_COOLDOWN:
        return
    last_play_times[string_index] = now
    snd = preloaded_sounds.get(string_index)
    if snd is None:
        
        fname = STRING_WAV_MAPPING.get(string_index)
        path = os.path.join(SOUNDS_DIR, fname) if fname else None
        if path and os.path.exists(path):
            try:
                snd = pygame.mixer.Sound(path)
                preloaded_sounds[string_index] = snd
            except Exception as e:
                print("Error loading sound on demand:", e)
                return
        else:
            
            return
    try:
        snd.play()
    except Exception as e:
        print("Error playing sound:", e)


fingertip_last_string = {}  


cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Could not open webcam.")

with mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.55, min_tracking_confidence=0.55) as hands:
    print("Controls: Press 1..7 or a..g to select chord A..G. Press 'p' to play/stop song. Press 'q' to quit.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to read frame.")
            break

        frame = cv2.flip(frame, 1)
        h, w, _ = frame.shape

        
        strum_top = int(h * STRUM_TOP_RATIO)
        strum_bottom = int(h * STRUM_BOTTOM_RATIO)
        string_positions = []
        for i in range(NUM_STRINGS):
            y = int(strum_top + (i / (NUM_STRINGS - 1)) * (strum_bottom - strum_top))
            string_positions.append(y)

        
        overlay = frame.copy()
        for i, y in enumerate(string_positions):
            cv2.line(overlay, (0, y), (w, y), STRING_COLORS[i], 4)

        
        cv2.rectangle(overlay, (0, strum_top - 4), (w, strum_bottom + 4), (0, 0, 0), 1)

        
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb)

        if results.multi_hand_landmarks and results.multi_handedness:
            for hand_idx, (hand_landmarks, handedness) in enumerate(zip(results.multi_hand_landmarks, results.multi_handedness)):
                mp_drawing.draw_landmarks(overlay, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                for lm_idx in FINGERTIP_LANDMARKS:
                    lm = hand_landmarks.landmark[lm_idx]
                    x_px = int(lm.x * w)
                    y_px = int(lm.y * h)

                    
                    cv2.circle(overlay, (x_px, y_px), 6, (0, 255, 0), -1)

                    
                    if y_px < strum_top - 25 or y_px > strum_bottom + 25:
                        current_string = None
                    else:
                        
                        distances = [abs(y_px - sy) for sy in string_positions]
                        current_string = int(np.argmin(distances))
                        spacing = (string_positions[1] - string_positions[0]) if NUM_STRINGS > 1 else 20
                        if distances[current_string] > (spacing * STRING_HIT_THRESHOLD_RATIO):
                            current_string = None

                    key = (hand_idx, lm_idx)
                    last_str = fingertip_last_string.get(key, None)

                    if current_string is not None and current_string != last_str:
                        
                        play_string(current_string)

                    fingertip_last_string[key] = current_string
        else:
           
            fingertip_last_string.clear()

        
        if song_mode and current_song is not None:
            elapsed = time.time() - song_start_time
            chord_name, duration = current_song[current_chord_index]
            if elapsed >= duration:
                current_chord_index += 1
                if current_chord_index >= len(current_song):
                    
                    song_mode = False
                    current_song = None
                    song_name = None
                else:
                    
                    song_start_time = time.time()
                    selected_chord = current_song[current_chord_index][0]

        
        def draw_chord_bubbles(img, chord_name, base_x=80, fret_spacing=36):
            """
            Draws chord bubbles near left side (base_x) for each string.
            - chord_name: chord key in CHORD_POSITIONS
            - base_x: leftmost x for open string / fret 0
            - fret_spacing: pixels per fret to the right
            """
            if chord_name not in CHORD_POSITIONS:
                return
            positions = CHORD_POSITIONS[chord_name]
            
            cv2.putText(img, f"Guide: {chord_name}", (base_x, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (240,240,0), 2)
            for i, fret in enumerate(positions):
                sx = base_x + (fret * fret_spacing if isinstance(fret, int) and fret > 0 else 0) if fret not in (None,) else base_x
                sy = string_positions[i]
               
                if fret is None:
                    
                    cv2.putText(img, "x", (base_x - 12, sy - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                elif fret == 0:
                   
                    cv2.circle(img, (base_x, sy), 10, (255,255,255), 2)
                else:
                    
                    cx = base_x + int(fret * fret_spacing)
                    cv2.circle(img, (cx, sy), 12, (0,200,0), -1)
                    
                    cv2.putText(img, str(fret), (cx-6, sy+6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)

        draw_chord_bubbles(overlay, selected_chord, base_x=70, fret_spacing=36)

        
        alpha = 0.75
        cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

        
        instr_w = int(w * 0.18)
        instr_h = int(h * 0.22)
        instr_img = chord_images.get(selected_chord)
        if instr_img is not None:
            ih, iw = instr_img.shape[:2]
            scale = min(instr_w / iw, instr_h / ih)
            new_w = int(iw * scale)
            new_h = int(ih * scale)
            resized = cv2.resize(instr_img, (new_w, new_h))
            
            frame[6:6+new_h, 6:6+new_w] = resized
        else:
            cv2.rectangle(frame, (6,6), (6 + instr_w, 6 + instr_h), (25,25,25), -1)
            cv2.putText(frame, f"{selected_chord}", (12, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)

        
        y_base = int(h * 0.88)
        cv2.putText(frame, "Press 1..7 or a..g to choose chord. 'p' Play/Stop song. 'q' Quit.", (10, y_base),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.55, (230,230,230), 1)

        if song_mode and current_song is not None:
            now = time.time()
            chord_name, duration = current_song[current_chord_index]
            elapsed = now - song_start_time
            remaining = max(0.0, duration - elapsed)
            next_chord = current_song[current_chord_index + 1][0] if (current_chord_index + 1) < len(current_song) else "-"
            cv2.putText(frame, f"Playing: {song_name}   Now: {chord_name}   Next: {next_chord}   ({remaining:.1f}s)", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2)
        else:
            cv2.putText(frame, f"Selected chord: {selected_chord}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200,200,200), 2)

        cv2.imshow("Virtual Guitar - chord guide (press q to quit)", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        if key in CHORD_KEYS:
            selected_chord = CHORD_KEYS[key]
            print("Selected chord:", selected_chord)
        if key == ord('p'):
            
            if not song_mode:
                
                song_name = "Simple" if "Simple" in SONGS else next(iter(SONGS.keys()))
                current_song = SONGS.get(song_name)
                if current_song:
                    song_mode = True
                    current_chord_index = 0
                    song_start_time = time.time()
                    selected_chord = current_song[0][0]
                    print(f"Started song: {song_name}")
            else:
                # stop
                song_mode = False
                current_song = None
                song_name = None
                print("Stopped song")

    

cap.release()
cv2.destroyAllWindows()
pygame.mixer.quit()



STRUM_AREA_TOP_RATIO = 0.45
STRUM_AREA_BOTTOM_RATIO = 0.55


BUBBLE_RADIUS = 16


STRING_HIT_THRESHOLD_RATIO = 0.6


STRING_COOLDOWN = 0.08   


STRING_WAV_MAPPING = {
    0: 'E3.wav',   
    1: 'A3.wav',
    2: 'D4.wav',
    3: 'G4.wav',
    4: 'B4.wav',
    5: 'E5.wav',   
}


STRING_COLORS = [
    (200, 80, 20),
    (0, 200, 0),
    (20, 20, 200),
    (200, 200, 0),
    (200, 0, 200),
    (0, 200, 200),
]


FINGERTIP_LANDMARKS = [4, 8, 12, 16, 20]


CHORD_POSITIONS = {
    "A": [None, 0, 2, 2, 2, 0],
    "C": [None, 3, 2, 0, 1, 0],
    "D": [None, None, 0, 2, 3, 2],
    "E": [0, 2, 2, 1, 0, 0],
    "G": [3, 2, 0, 0, 0, 3],
    "Em": [0, 2, 2, 0, 0, 0],
    "Am": [None, 0, 2, 2, 1, 0],
}


preloaded_sounds = {}
for idx, fname in STRING_WAV_MAPPING.items():
    p = os.path.join(SOUNDS_DIR, fname)
    if os.path.exists(p):
        try:
            preloaded_sounds[idx] = pygame.mixer.Sound(p)
        except Exception as e:
            print(f"[WARN] Could not load sound {p}: {e}")
    else:
        print(f"[WARN] Missing sound: {p}")

if len(preloaded_sounds) < NUM_STRINGS:
    print("[WARN] Some string sounds missing. Expected:", list(STRING_WAV_MAPPING.values()))


def load_chord_image(ch):
    path = os.path.join(CHORD_IMG_DIR, f"{ch}.png")
    if os.path.exists(path):
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        if img is None:
            return None
        if img.shape[2] == 4:
            alpha = img[:, :, 3] / 255.0
            img_rgb = img[:, :, :3].astype(np.float32)
            for c in range(3):
                img[:, :, c] = (alpha * img_rgb[:, :, c] + (1 - alpha) * 255).astype(np.uint8)
            img = img[:, :, :3]
        return img
    return None

chord_images = {c: load_chord_image(c) for c in chords}


last_play_times = [0.0] * NUM_STRINGS

def play_string(idx):
    """Play a string if cooldown passed."""
    now = time.time()
    if now - last_play_times[idx] < STRING_COOLDOWN:
        return
    last_play_times[idx] = now
    snd = preloaded_sounds.get(idx)
    if snd:
        snd.play()
    else:
        
        fname = STRING_WAV_MAPPING.get(idx)
        if fname:
            path = os.path.join(SOUNDS_DIR, fname)
            if os.path.exists(path):
                try:
                    s = pygame.mixer.Sound(path)
                    preloaded_sounds[idx] = s
                    s.play()
                except Exception as e:
                    print("Could not play sound on demand:", e)


held_by = {i: set() for i in range(NUM_STRINGS)}


fingertip_prev_y = {}   


fingertip_on_bubble = {}  


cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Could not open webcam.")

with mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.55, min_tracking_confidence=0.55) as hands:
    print("Controls: 1..7/a..g select chord (for guidance). q quit.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to read frame.")
            break

        frame = cv2.flip(frame, 1)
        H, W, _ = frame.shape

        
        strum_top = int(H * STRUM_TOP_RATIO)
        strum_bottom = int(H * STRUM_BOTTOM_RATIO)
        string_positions = [int(strum_top + (i / (NUM_STRINGS - 1)) * (strum_bottom - strum_top)) for i in range(NUM_STRINGS)]

        
        area_top = int(H * STRUM_AREA_TOP_RATIO)
        area_bottom = int(H * STRUM_AREA_BOTTOM_RATIO)

       
        overlay = frame.copy()
        for i, y in enumerate(string_positions):
            cv2.line(overlay, (0, y), (W, y), STRING_COLORS[i], 4)

        
        cv2.rectangle(overlay, (0, area_top), (W, area_bottom), (40, 40, 40), 1)
        cv2.putText(overlay, "STRUM AREA", (W - 160, area_top + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (180,180,180), 1)

        
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb)

        
        for i in range(NUM_STRINGS):
            held_by[i].clear()
        fingertip_current_on = {}  

        if results.multi_hand_landmarks and results.multi_handedness:
            for hand_idx, (hand_landmarks, handedness) in enumerate(zip(results.multi_hand_landmarks, results.multi_handedness)):
                mp_drawing.draw_landmarks(overlay, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                for lm_idx in FINGERTIP_LANDMARKS:
                    lm = hand_landmarks.landmark[lm_idx]
                    x_px = int(lm.x * W)
                    y_px = int(lm.y * H)
                    key = (hand_idx, lm_idx)

                    
                    cv2.circle(overlay, (x_px, y_px), 5, (0,255,0), -1)

                    
                    base_x = int(W * 0.10)
                    fret_spacing = 34  
                    bubble_hit = None
                    if selected_chord in CHORD_POSITIONS:
                        chord_shape = CHORD_POSITIONS[selected_chord]
                        for s_idx, fret in enumerate(chord_shape):
                           
                            if fret is None:
                                cx = base_x
                                cy = string_positions[s_idx]
                            elif fret == 0:
                                cx = base_x
                                cy = string_positions[s_idx]
                            else:
                                cx = base_x + int(fret * fret_spacing)
                                cy = string_positions[s_idx]
                            if (x_px - cx) ** 2 + (y_px - cy) ** 2 <= (BUBBLE_RADIUS * 1.1) ** 2:
                                bubble_hit = s_idx
                                break

                    fingertip_current_on[key] = bubble_hit
                    
                    if bubble_hit is not None:
                        held_by[bubble_hit].add(key)

                    
                    prev_y = fingertip_prev_y.get(key, None)
                    
                    if prev_y is None:
                        fingertip_prev_y[key] = y_px
                        prev_y = y_px

                    
                    crossed = False
    
                    if prev_y < area_top and y_px >= area_top:
                        crossed = True
                        direction = "down"
                    
                    elif prev_y < area_bottom and y_px >= area_bottom:
                        crossed = True
                        direction = "down"
                    elif prev_y > area_bottom and y_px <= area_bottom:
                        crossed = True
                        direction = "up"
                    elif prev_y > area_top and y_px <= area_top:
                        crossed = True
                        direction = "up"
                    else:
                        direction = None

                    if crossed:
                        
                        low = min(prev_y, y_px)
                        high = max(prev_y, y_px)
                        
                        strings_crossed = []
                        for s_idx, s_y in enumerate(string_positions):
                            if low - 1 <= s_y <= high + 1:
                                strings_crossed.append((s_idx, s_y))
                        
                        strings_crossed.sort(key=lambda t: t[1], reverse=(direction=="up"))
                        
                        for s_idx, _ in strings_crossed:
                            if len(held_by[s_idx]) > 0:
                                play_string(s_idx)
                        
                    
                    fingertip_prev_y[key] = y_px

        else:
            
            fingertip_prev_y.clear()
            fingertip_current_on.clear()
            for i in range(NUM_STRINGS):
                held_by[i].clear()

        
        def draw_chord_bubbles(img, chord_name, base_x_ratio=0.10, fret_spacing=34):
            base_x = int(W * base_x_ratio)
            if chord_name not in CHORD_POSITIONS:
                return
            positions = CHORD_POSITIONS[chord_name]
            
            cv2.putText(img, f"Guide: {chord_name}", (base_x - 10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (240,240,0), 2)
            for i, fret in enumerate(positions):
                sy = string_positions[i]
                if fret is None:
                    cx = base_x
                    
                    cv2.putText(img, "x", (cx - 10, sy - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
                elif fret == 0:
                    cx = base_x
                    cv2.circle(img, (cx, sy), BUBBLE_RADIUS, (255,255,255), 2)
                else:
                    cx = base_x + int(fret * fret_spacing)
                    
                    if len(held_by[i]) > 0:
                        cv2.circle(img, (cx, sy), BUBBLE_RADIUS, (0,200,0), -1)
                        cv2.putText(img, str(fret), (cx-6, sy+6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
                    else:
                        cv2.circle(img, (cx, sy), BUBBLE_RADIUS, (0,200,0), 2)
                        cv2.putText(img, str(fret), (cx-6, sy+6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,200,0), 1)

        draw_chord_bubbles(overlay, selected_chord, base_x_ratio=0.10, fret_spacing=34)

        
        alpha = 0.78
        cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

        
        instr_w = int(W * 0.18)
        instr_h = int(H * 0.22)
        instr_img = chord_images.get(selected_chord)
        if instr_img is not None:
            ih, iw = instr_img.shape[:2]
            scale = min(instr_w / iw, instr_h / ih)
            new_w = int(iw * scale)
            new_h = int(ih * scale)
            resized = cv2.resize(instr_img, (new_w, new_h))
            frame[6:6+new_h, 6:6+new_w] = resized
        else:
            cv2.rectangle(frame, (6,6), (6 + instr_w, 6 + instr_h), (25,25,25), -1)
            cv2.putText(frame, f"{selected_chord}", (12, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)

        # UI text
        cv2.putText(frame, "Place finger on bubble to hold string. Sweep finger through STRUM AREA to play held strings.",
                    (10, H - 18), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (220,220,220), 1)

        cv2.imshow("Virtual Guitar - press q to quit", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        if key in CHORD_KEYS:
            selected_chord = CHORD_KEYS[key]
            print("Selected chord:", selected_chord)


cap.release()
cv2.destroyAllWindows()
pygame.mixer.quit()


I0000 00:00:1760118013.141848    5921 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1760118013.146106    7051 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 25.0.7-0ubuntu0.24.04.1), renderer: AMD Radeon Vega 3 Graphics (radeonsi, raven2, ACO, DRM 3.61, 6.14.0-29-generic)
W0000 00:00:1760118013.179457    7046 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1760118013.229019    7046 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Controls: 1..7/a..g select chord (for guidance). q quit.
